# Assignment 2: recommender System

## Code from the lecture

In [ ]:
#read data to DataFrames
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../../DATA/movielens100k/u.user', sep='|', names=u_cols, encoding = "ISO-8859-1")

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../../DATA/movielens100k/u.data', sep='\t', names=r_cols, encoding = "ISO-8859-1")

m_cols=['movie_id', 'title', 'release date', 'video release date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('../../DATA/movielens100k/u.item', sep='|', names=m_cols ,encoding = "ISO-8859-1" )

In [ ]:
#gerate matrix (this can be done more efficiently!)
R=np.zeros((np.shape(users)[0],np.shape(movies)[0]))
for i in range(np.shape(ratings)[0]):
    R[ratings['user_id'][i]-1, ratings['movie_id'][i]-1]=ratings['rating'][i]

In [ ]:
def CosineDist(a,b):
    return a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [ ]:
#again, implementation not efficient: better use lamda opperators 
D=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            D[i,j]=CosineDist(R[:,i],R[:,j])
np.save("movie_file_1",D)


In [ ]:
def getTopN(movie_id,N=5):
    return D[movie_id,:].argsort()[-N:]

def getIDbyName(name):
    if np.size(movies.movie_id[movies.title.str.contains(name)]) > 0:
        m = int(movies.movie_id[movies.title.str.contains(name)][:1]), str(movies.title[movies.title.str.contains(name)][:1])
        return m[0]-1
    else:
        return -1
    
def getNameByID(IDs):
    res=movies.iloc[IDs]
    return res.title

def CII(title):
    if getIDbyName(title) > 0:
        print ("recommending movies for: '" + str(getNameByID(getIDbyName(title)))+"'")
        return getNameByID(getTopN(getIDbyName(title)))[::-1]
    else:
        print ("no movie title containing " + str(title) + "found...")
        

def Score_byID(ID, D, Test):
    #print ("Hit Scores for: ", getNameByID(ID))
    res_id = getTopN(ID,D)[::-1]
    res_title = getNameByID(getTopN(ID,D))[::-1]
    res_score = Test[res_id]
    return res_id, res_title, res_score, np.mean(res_score)

def test_Score(D_train, R_test):
    userScores=[]
    for i in range(R_test.shape[0]):
        userScore=0
        userTop = np.argsort(R_test[i])[-5:]
        print(type(userTop))
        for e in userTop:
            res_id, res_title, res_score, av_score = Score_byID(e,D_train, R_test[i])
            userScore+=av_score
        userScores.append(userScore/(5))
    return userScores



## 4.1 Alternative Distance Measures
Implement a different distance measure, e.g. eucledian and compare the ressults.

In [ ]:
#Euclidische Distanz

In [ ]:
def EuclDist(a,b):
    return np.linalg.norm(a-b)

In [ ]:
#Erstellung Euclidische-Distanz Matrix

In [ ]:
ED = np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            ED[i,j]=EuclDist(R[:,i],R[:,j])
#np.save("movie_file_2",ED)

In [ ]:
#cosinus-Distanz6
D=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            D[i,j]=CosineDist(R[:,i],R[:,j])

#Euclidische Distanz-Matrix darstellen

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.matshow(ED)
plt.xlabel('movie id')
plt.ylabel('user id ')
plt.colorbar() 
plt.show()

In [ ]:
#Cosinus Distanz-Matrix darstellen

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.matshow(D)
plt.xlabel('movie id')
plt.ylabel('user id ')
plt.colorbar() 
plt.show()

In [ ]:
#Auswertung

In [ ]:
#Ausgangsdaten in Training und Testdaten 
#Frage: Warum sind die Testdaten so klein gehalten? bewusst?
from sklearn.model_selection import train_test_split
R_train, R_test = train_test_split(R, test_size=0.1)
plt.matshow(R_test)
plt.xlabel('Film ID')
plt.ylabel('User ID')
plt.colorbar()
plt.show()

In [ ]:
D[getIDbyName('Star Wars'),:].argsort()[-5:]

In [ ]:
#print(ED_train.shape)
#print(ED_train)
#print(D_train.shape)
#print(ED_train)
#print(R_test.shape)
#print(R_train.shape)

In [ ]:
#Anfrage auf dem Trainingsdatensatz
CII("Star")


In [ ]:
#CII("Star",ED_train)

In [ ]:
#Test Score Cosinus-Distanz

In [ ]:
#D_train_Arr=np.array(D_train)
#R_Test_Arr=np.array(R_test)

result_ED = test_Score(ED_train, R_test)

#resultD = test_Score(D_train, R_test)



In [ ]:
plt.hist(resultD)

In [ ]:
#Test Score  Euclidische Distanz

In [ ]:
resultED = test_Score(ED_train, R_test)

In [ ]:
plt.hist(resultED)

In [ ]:
#Mean Squared Error

In [ ]:
error_Cosin= np.mean(resultD)
print("Mean Squared Error der Cosinus-Distanz: ",error_Cosin)

In [ ]:
error_Eucl = np.mean(resultED)
print("Mean Squared Error der Euclidischen-Distanz: ",error_Eucl)

## 4.2 Baseline Results
*Implement and compare results of
* random recommendations
* allways recommending the top 5 movies (over all users)

In [ ]:
#print (np.shape(users), np.shape(ratings), np.shape(movies))
#print (ratings[1:10])


In [ ]:
#5 top movie

In [ ]:
#getTopN(movie_id,movie)

#type(D)
#type(ED)

#Random Recomondation
#random_indices = np.random.choice(5, 5)
#print(D[random_indices, :])
#

In [ ]:
CII("gr")

In [ ]:
CII("High")

In [ ]:
CII("Poke")

In [ ]:
#def getTopN(movie_id,N=5):
   # return print(D[movie_id,:].argsort()[-N:])

#getTopN(D[ratings['movie_id']])

#print((D[ratings['movie_id']]))



In [ ]:
#Ausgangsdaten in Training und Testdaten 
#Frage: Warum sind die Testdaten so klein gehalten? bewusst?
from sklearn.model_selection import train_test_split
R_train, R_test = train_test_split(R, test_size=0.1)
plt.matshow(R_test)
plt.xlabel('Film ID')
plt.ylabel('User ID')
plt.colorbar()
plt.show()

In [ ]:
#cosinus-Distanz6
D_train=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            D_train[i,j]=CosineDist(R_train[:,i],R_train[:,j])



In [ ]:
#Euclidische Distanz
ED_train=np.zeros((np.shape(movies)[0],np.shape(movies)[0]))
for i in range(0,np.shape(movies)[0]):
    for j in range(0,np.shape(movies)[0]):
        if i!=j:
            ED_train[i,j]=EuclDist(R_train[:,i],R_train[:,j])

In [ ]:
#Anfrage auf dem Trainingsdatensatz
CII("Star",D_train)
CII("Star",ED_train)

In [ ]:
#get top 5 ids from random test user 23
np.argsort(R_test[23])[-5:]

In [ ]:
#Movie ID Ratings eines bestimmten nutzers?
Score_byID(326,D_train, R_test[23])
Score_byID(326,ED_train, R_test[23])

In [ ]:
test_res_D=test_Score(D_train,R_test)
test_res_ED = test_Score(ED_train,R_test)

In [ ]:
np.mean(test_res_D)

## 4.3 SVD
Implement the SVD approach. Hints: 
* use numpy.linalg.svd()
* apply SVD on $D$
* plot the singular values and decide where to cut off the reconstruction
* reconstruct and evaluate $D'$


In [ ]:
print(D)
print(D.shape)
U,S,V = np.linalg.svd(D)


In [ ]:
print(U)

In [ ]:
#von allen Zeilen nur die ersten 3 werte
print(U[:,:3])
print(U[:,:3].shape)

In [ ]:
S

In [ ]:
V

In [ ]:
#D wieder serstellem
print(D)


#print(U[:,:3])

In [ ]:

#M[lower:upper:step]
np.dot(U[:,:3]*S[:3],V[:3,:6])